# Mixing Ingredients—Python Dictionaries
<img src="misc/4_ingredients on table and mixing bowl.png" width="100%" />

In Lesson 2 we read the ingredients/steps.  In Lesson 3, we parsed those ingredients to gain some understanding of what they do. Now we will take the steps and mix the ingredients. In doing so we will gain a deeper understanding of advanced data structures, like dictionaries.

In this chapter we read over the steps and operate on the ingredients.

> read in the steps

In [1]:
import pickle
recipe_as_dict = pickle.load(open("data.pickle", "rb"))

## 4.1 Logging and control

> Our program is getting more complex; we will start using the built-in Python logging to help us get feedback as we run.

In [2]:
import logging as log
from pprint import pformat
logger = log.getLogger()
logger.setLevel(log.DEBUG)

In [3]:
ch = log.StreamHandler()
ch.setFormatter(log.Formatter('%(levelname)s: %(message)s'))
logger.removeHandler(0)
logger.addHandler(ch)
log.info("who loves Pie?")
log.debug("i am a debug message")

INFO: who loves Pie?
DEBUG: i am a debug message


Let's break out the filling and crust steps:

In [4]:
#let's make a list of filling 
filling_steps = None
crust_steps = None
for part in recipe_as_dict["Parts"]:
    if 'filling' == part['sub-title']:
        filling_steps = part['steps']
    elif 'crust' == part['sub-title']:
        crust_steps = part['steps']
log.info("{} filling steps and {} crust steps".format(len(filling_steps),
                                                      len(crust_steps)))

INFO: 3 filling steps and 6 crust steps


## 4.2 Start with the filling

Inspect the filling_steps:

In [5]:
log.info(pformat(filling_steps))
log.info(len(filling_steps))

INFO: ['Line pie plate with unbaked pastry.  Mix together the sugars, flour\n'
 'and spices.  Pour 1/2 of this mixture into the pie plate, smoothing\n'
 'evenly across the bottom.  Lay the apples in the crust, enough so that\n'
 "they reach the top of the plate but aren't mounded over it.  Sprinkle\n"
 'the remaining flour mixture over the top of the apples.  Sprinkle 1\n'
 'tablespoon of lemon juice over the top, then dot with 1 tablespoon of\n'
 'butter.  Add the top crust and cut slits for steam.',
 'Bake 15 minutes at 425 degrees, then reduce heat to 350 and bake\n'
 'another hour.  The trick with apple pie is to be sure and bake it long\n'
 'enough for the apples to be soft.  I usually stick a fork inside one\n'
 'of the slits to make sure, but an hour should be plenty.  If the edge\n'
 'of the crust starts to brown too much, cover it loosely with foil.',
 "It's a good idea to set the pie on a cookie sheet in case the apples\n"
 'are particularly juicy and spill over into the oven

A couple of observations:

 * There are three parts
 * There is a bunch of `\n`, which means line break, but they aren't important
 * What may be important is the sentences in each

Let's break this up into a list of list based on sentences.


In [6]:
list_of_list_of_steps = []
for step in filling_steps:
    step = step.replace("\n", " ")
    list_of_list_of_steps.append(step.split(". "))
    log.debug("step")
list_of_list_of_steps

DEBUG: step
DEBUG: step
DEBUG: step


[['Line pie plate with unbaked pastry',
  ' Mix together the sugars, flour and spices',
  ' Pour 1/2 of this mixture into the pie plate, smoothing evenly across the bottom',
  " Lay the apples in the crust, enough so that they reach the top of the plate but aren't mounded over it",
  ' Sprinkle the remaining flour mixture over the top of the apples',
  ' Sprinkle 1 tablespoon of lemon juice over the top, then dot with 1 tablespoon of butter',
  ' Add the top crust and cut slits for steam.'],
 ['Bake 15 minutes at 425 degrees, then reduce heat to 350 and bake another hour',
  ' The trick with apple pie is to be sure and bake it long enough for the apples to be soft',
  ' I usually stick a fork inside one of the slits to make sure, but an hour should be plenty',
  ' If the edge of the crust starts to brown too much, cover it loosely with foil.'],
 ["It's a good idea to set the pie on a cookie sheet in case the apples are particularly juicy and spill over into the oven."]]

We want to identify the keywords that indicate mixing:

 * "mix"
 * "pour"
 * "sprinkle"
 * "dot"

In [7]:
known_operations = ("mix ", "pour ", "sprinkle ", "dot ")

From Lesson 3, we have a list of ingredients already broken up into quantities and units.

The ingredients will live in three states:

 * The preparation counter (the_counter)
 * The mixing bowl (mixing_bowl)
 * The pie plate (pie_plate)
 
We need to move ingredients from state to state from step to step and keep track of the ingredients. Later we will need to interrupt the pie-making process by refilling the ingredients on the preparation counter.

> We start out with those steps and identify which ones require action.

In [8]:
the_counter = {}
mixing_bowl = {}
pie_plate = {}



We need to break it out into functions:

 * A function to test whether it has a mix operation (has_mix_operation)
 * A function to do the mixing (mix)
 
Before we pick it up with the actual steps, we prototype each function.


In [9]:
def has_mix_operation(step):
    "test if the step needs a mix operation. Returns: True or False"
    for operation in known_operations:
        if operation in step.lower():
            log.debug("'{}' found in '{}'".format(operation, step))
            return True
    log.warning("'{}' not found in '{}'".format(known_operations, step))
    return False

In [10]:
has_mix_operation("Line pie plate with unbaked pastry")

False

In [11]:
has_mix_operation("Mix together the sugars, flour and spices")

DEBUG: 'mix ' found in 'Mix together the sugars, flour and spices'


True

The `mix()` function is what does the mixing (duh), but it also uses dictionaries (dicts) as input. But first a quick tutorial on dicts.

## 4.3 Notes on dicts

A couple of notes on dictionaries:

 * Dicts are not sorted, so they don't retain order
 * Keys must be unique
 * Dicts point to data values so they retain changes
 * The types used for keys and values do not need to be same even inside same dict

Some examples of this...

```python
>>> the_dict = {'a':2, 'b':2, 'c':2, 'd':4, 'e':4}
>>> print(the_dict)
{'d': 4, 'b': 2, 'c': 2, 'a': 2, 'e': 4}
>>> def changeone(passed_dict):
...    passed_dict['d'] = 1337
>>> changeone(the_dict)
>>> print(the_dict)
{'d': 1337, 'b': 2, 'c': 2, 'a': 2, 'e': 4}
```

Now back to our `mix()` function:

In [12]:
from output import LargeItem, IngredientBase, DrySolid, Liquid
from fractions import Fraction

In [13]:
import pickle
shopping_list = pickle.load(open("shopping_list.pickle", "rb"))

In [14]:
shopping_list

[<Ingredient (large item): Granny Smith apples, depending on size, peeled and sliced - 3 item>,
 <Ingredient (solid): brown sugar - 1/2 cup>,
 <Ingredient (solid): granulated sugar - 1/2 cup>,
 <Ingredient (solid): flour - 1/4 cup>,
 <Ingredient (liquid): apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg) - 1 tsp>,
 <Ingredient (solid): flour - 2 cups>,
 <Ingredient (solid): salt - 1 tsp>,
 <Ingredient (solid): solid shortening (like Crisco) - 3/4 cup>,
 <Ingredient (liquid): ice water - 1/4 cup>]

In [15]:
for x in shopping_list: print(x.__dict__)

{'qty_max': '4', 'qty': Fraction(3, 1), 'original_ingredient_str': '3-4 Granny Smith apples, depending on size, peeled and sliced', 'unit': 'item', 'item': 'Granny Smith apples, depending on size, peeled and sliced'}
{'qty_max': 0, 'qty': Fraction(1, 2), 'original_ingredient_str': '1/2 cup brown sugar', 'unit': 'cup', 'item': 'brown sugar'}
{'qty_max': 0, 'qty': Fraction(1, 2), 'original_ingredient_str': '1/2 cup granulated sugar', 'unit': 'cup', 'item': 'granulated sugar'}
{'qty_max': 0, 'qty': Fraction(1, 4), 'original_ingredient_str': '1/4 cup flour', 'unit': 'cup', 'item': 'flour'}
{'qty_max': 0, 'qty': Fraction(1, 1), 'original_ingredient_str': '1 tsp apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)', 'unit': 'tsp', 'item': 'apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)'}
{'qty_max': 0, 'qty': Fraction(2, 1), 'original_ingredient_str': '2 cups flour', 'unit': 'cups', 'item': 'flour'}
{'qty_max': 0, 'qty': Fraction(1, 1), 'original_ingredient_str': '1 tsp salt', 'uni

## 4.4 Reusable code

In [16]:
def move_items(items, from_dict, to_dict, qty=0.0, move_all=False):
    """move items from_dict to to_dict in qty or move_all to move it all.
    """
    for item in items:
    
        if move_all:
            qty = from_dict[item].qty
            log.debug("moving all {}".format(qty))
        else:
            log.debug("moving {}".format(qty))
    
        # assignments in dicts stick
        from_dict[item].qty -= qty
        to_dict[item].qty += qty
        
    
# test moving parts    
start = {"beer": Liquid("6 cups of beer")}
end = {"beer": Liquid("0 cups of beer")}
move_items(["beer", ], start, end, qty=3)
assert start["beer"].qty == Fraction("3")
assert end["beer"].qty == Fraction("3")

# test moving all
move_items(["beer", ], start, end, move_all=True)

assert start["beer"].qty == Fraction("0")
assert end["beer"].qty == Fraction("6")

DEBUG: moving 3
DEBUG: moving all 3


In [17]:
the_counter = {}
mixing_bowl = {}
pie_plate = {}

for item in shopping_list:
    the_counter[item.item] = item.copy()
    mixing_bowl[item.item] = item.empty()
    pie_plate[item.item] = item.empty()

the_counter, mixing_bowl, pie_plate

({'Granny Smith apples, depending on size, peeled and sliced': <Ingredient (large item): Granny Smith apples, depending on size, peeled and sliced - 3 item>,
  'apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)': <Ingredient (liquid): apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg) - 1 tsp>,
  'brown sugar': <Ingredient (solid): brown sugar - 1/2 cup>,
  'flour': <Ingredient (solid): flour - 2 cups>,
  'granulated sugar': <Ingredient (solid): granulated sugar - 1/2 cup>,
  'ice water': <Ingredient (liquid): ice water - 1/4 cup>,
  'salt': <Ingredient (solid): salt - 1 tsp>,
  'solid shortening (like Crisco)': <Ingredient (solid): solid shortening (like Crisco) - 3/4 cup>},
 {'Granny Smith apples, depending on size, peeled and sliced': <Ingredient (large item): Granny Smith apples, depending on size, peeled and sliced - 0 item>,
  'apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)': <Ingredient (liquid): apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg) - 0 tsp>,
 

In [18]:
def mix(the_counter, mixing_bowl, step):
    """Takes the different work areas as dictionaries.
       Inspects the step.
       Adjusts the quantities
       """
    target_items = []
    for item in shopping_list:
        if item.does_match_target(step):
            log.debug("{} matches".format(item))
            target_items.append(item.item)
            
    if "together" in step:
        move_items(target_items, the_counter, mixing_bowl, move_all=True)
    
test_step = "Mix together the sugars, flour and spices"

mix(the_counter, mixing_bowl, test_step)


DEBUG: <Ingredient (solid): brown sugar - 1/2 cup> matches
DEBUG: <Ingredient (solid): granulated sugar - 1/2 cup> matches
DEBUG: <Ingredient (solid): flour - 1/4 cup> matches
DEBUG: <Ingredient (liquid): apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg) - 1 tsp> matches
DEBUG: <Ingredient (solid): flour - 2 cups> matches
DEBUG: <Ingredient (solid): salt - 1 tsp> matches
DEBUG: <Ingredient (solid): solid shortening (like Crisco) - 3/4 cup> matches
DEBUG: <Ingredient (liquid): ice water - 1/4 cup> matches
DEBUG: moving all 1/2
DEBUG: moving all 1/2
DEBUG: moving all 2
DEBUG: moving all 1
DEBUG: moving all 0
DEBUG: moving all 1
DEBUG: moving all 3/4
DEBUG: moving all 1/4


## 4.5 Putting it all together.

 * Fill the counter with items and move to the mixing_bowl (pie_plate will remain empty for now)
 * Iterate over each steps
 * Operate properly on each




In [19]:
the_counter = {}
mixing_bowl = {}
pie_plate = {}

for item in shopping_list:
    the_counter[item.item] = item.copy()
    mixing_bowl[item.item] = item.empty()
    pie_plate[item.item] = item.empty()
    
    
for step_part in list_of_list_of_steps:
    for step in step_part:
        if has_mix_operation(step):
            log.debug("mixing")
            mix(the_counter, mixing_bowl, step)
            break
    break


DEBUG: 'mix ' found in ' Mix together the sugars, flour and spices'
DEBUG: mixing
DEBUG: <Ingredient (solid): brown sugar - 1/2 cup> matches
DEBUG: <Ingredient (solid): granulated sugar - 1/2 cup> matches
DEBUG: <Ingredient (solid): flour - 1/4 cup> matches
DEBUG: <Ingredient (liquid): apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg) - 1 tsp> matches
DEBUG: <Ingredient (solid): flour - 2 cups> matches
DEBUG: <Ingredient (solid): salt - 1 tsp> matches
DEBUG: <Ingredient (solid): solid shortening (like Crisco) - 3/4 cup> matches
DEBUG: <Ingredient (liquid): ice water - 1/4 cup> matches
DEBUG: moving all 1/2
DEBUG: moving all 1/2
DEBUG: moving all 2
DEBUG: moving all 1
DEBUG: moving all 0
DEBUG: moving all 1
DEBUG: moving all 3/4
DEBUG: moving all 1/4


In [20]:
the_counter


{'Granny Smith apples, depending on size, peeled and sliced': <Ingredient (large item): Granny Smith apples, depending on size, peeled and sliced - 3 item>,
 'apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)': <Ingredient (liquid): apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg) - 0 tsp>,
 'brown sugar': <Ingredient (solid): brown sugar - 0 cup>,
 'flour': <Ingredient (solid): flour - 0 cups>,
 'granulated sugar': <Ingredient (solid): granulated sugar - 0 cup>,
 'ice water': <Ingredient (liquid): ice water - 0 cup>,
 'salt': <Ingredient (solid): salt - 0 tsp>,
 'solid shortening (like Crisco)': <Ingredient (solid): solid shortening (like Crisco) - 0 cup>}

In [21]:
mixing_bowl

{'Granny Smith apples, depending on size, peeled and sliced': <Ingredient (large item): Granny Smith apples, depending on size, peeled and sliced - 0 item>,
 'apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg)': <Ingredient (liquid): apple pie spice (or 1 tsp cinnamon and 1/2 tsp nutmeg) - 1 tsp>,
 'brown sugar': <Ingredient (solid): brown sugar - 1/2 cup>,
 'flour': <Ingredient (solid): flour - 2 cups>,
 'granulated sugar': <Ingredient (solid): granulated sugar - 1/2 cup>,
 'ice water': <Ingredient (liquid): ice water - 1/4 cup>,
 'salt': <Ingredient (solid): salt - 1 tsp>,
 'solid shortening (like Crisco)': <Ingredient (solid): solid shortening (like Crisco) - 3/4 cup>}

## 4.6 Code-reuse

We can cut and past the code above and apply to crust. We will find out in the following chapter how this code can better be organized as functions.

In [22]:
list_of_list_of_steps = []
for step in crust_steps:  #<- we only changed this to crust_steps
    step = step.replace("\n", " ")
    list_of_list_of_steps.append(step.split(". "))
    log.debug("step")
    
for step_part in list_of_list_of_steps:
    for step in step_part:
        if has_mix_operation(step):
            log.debug("mixing")
            mix(the_counter, mixing_bowl, step)
            break
    break

DEBUG: step
DEBUG: step
DEBUG: step
DEBUG: step
DEBUG: step
DEBUG: step
DEBUG: 'mix ' found in 'Mix together the flour and the salt'
DEBUG: mixing
DEBUG: <Ingredient (solid): brown sugar - 1/2 cup> matches
DEBUG: <Ingredient (solid): granulated sugar - 1/2 cup> matches
DEBUG: <Ingredient (solid): flour - 1/4 cup> matches
DEBUG: <Ingredient (solid): flour - 2 cups> matches
DEBUG: <Ingredient (solid): salt - 1 tsp> matches
DEBUG: <Ingredient (solid): solid shortening (like Crisco) - 3/4 cup> matches
DEBUG: moving all 0
DEBUG: moving all 0
DEBUG: moving all 0
DEBUG: moving all 0
DEBUG: moving all 0
DEBUG: moving all 0
